### Chapter 13 - Custom User Model:

The biggest problem with the auth.user is that it is very hard to modify once you migrate, also in django documentation it is said that we should avoid any changes.<br>

The solution is to use a CustomUser model.<br>
Please understand that in order to use a CustomUser model you should be in the early stage of you project, meaning that you shouldnt have done any migrations otherwise it is almost imposible to achieve.<br>

So, lets start with a new project:

### BookStore:
1. Create a new django project called 'BookStore'
2. Add a new app called 'accounts' to the project
3. Add the new add to the INSTALLED_APP list (settings.py)
4. Navigate to account.models.py
5. From django.contrib.auth.models import AbstractUser
6. Create a new model called CustomUser and extend it from AbstractUser class

In [ ]:
from django.contrib.auth.models import AbstractUser

class CustomUser(AbstractUser):
    pass

Now we need to tell django that it should use this model instead of its existing auth model

7. Navigate to settings.py and add a new variable called AUTH_USER_MODEL and pass 'accounts.CustomUser' to it

In [ ]:
AUTH_USER_MODEL = 'accounts.CustomUser'